In [3]:
import pickle
import numpy as np
from tqdm import tqdm

In [5]:
all_cls = pickle.load(open('/data/jimenezgutierb2/umls2020_sapbert_vecs.p','rb'))

In [4]:
import torch
import transformers
from transformers import AutoModel, AutoTokenizer

In [5]:
pt_model = 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext'
model = AutoModel.from_pretrained(pt_model)
model.to('cuda')

tokenizer = AutoTokenizer.from_pretrained(pt_model)

In [3]:
# original_umls_2020, new_umls_2020 = pickle.load(open('aui_string_map_UMLS2020_update.p','rb'))

In [43]:
# aui_strings = [t[1] for t in original_umls_2020]
# auis = [t[0] for t in original_umls_2020]
# aui_strings.extend([t[1] for t in new_umls_2020])
# auis.extend([t[0] for t in new_umls_2020])

In [18]:
lengths = []

for text_batch in tqdm(aui_strings):
    encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=False)
    input_ids = encoding['input_ids']
    
    lengths.append(len(input_ids[0]))

100%|████████████████████████████████████████████████████████████████████████████████████| 8951355/8951355 [1:06:36<00:00, 2239.59it/s]


In [19]:
# pickle.dump(lengths, open('umls_string_lengths.p','wb'))

In [6]:
import pandas as pd

In [23]:
lengths_df = pd.DataFrame(lengths)

In [44]:
lengths_df['strings'] = aui_strings
lengths_df['auis'] = auis

In [55]:
sorted_umls_df = lengths_df.sort_values(0)

In [47]:
long_indices = lengths_df[lengths_df[0] >= 50].sort_values(0)

In [56]:
# sorted_umls_df.to_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/sorted_umls2020_auis.csv',sep='\t')

In [8]:
sorted_umls_df = pd.read_csv('/data/Bodenreider_UMLS_DL/Interns/Bernal/sorted_umls2020_auis.csv',sep='\t')

In [9]:
aui_strings = sorted_umls_df.strings

In [14]:
text_batch = aui_strings[:200]
text_batch = list(text_batch)
encoding = tokenizer(text_batch, return_tensors='pt', padding='max_length', truncation=True, max_length=60)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

In [15]:
input_ids.shape

torch.Size([200, 60])

In [16]:
input_ids = input_ids.to('cuda')
attention_mask = attention_mask.to('cuda')

outputs = model(input_ids, attention_mask=attention_mask)

In [21]:
input_ids = None
attention_mask = None
outputs = None

In [23]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sun Jun 12 21:07:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:8B:00.0 Off |                  Off |
| N/